In [32]:
include("../Algebra/Gradients.jl")
include("../Algebra/Hamiltonians.jl")
include("../Algebra/Matrices.jl")
include("../Algebra/Propagators.jl")
include("../Amplitudes/Chebyshev.jl")
include("../Costs/Costs.jl")
include("../Gates/Xgate.jl")

U2 (generic function with 1 method)

In [33]:
using LinearAlgebra, Optim, Measures, CSV, DataFrames,Plots

In [41]:
ω_c = 5000
ω_q = 3000
χ = 3e-1
N = 4
amp_c = 1
amp_q = 1
T = 0.1

0.1

### Target gate

In [42]:
I_qubit = Matrix(I,2,2)
I_cavity = Matrix(I,N,N)
#X_gate = kron(x_gate(N, Array[[3,4], [5,6], [7,8]]), I_qubit)
#X_gate = kron(x_gate(N, Array[[1,2]]), I_qubit)
X_gate = kron(x_gate(N, Array[[3,4]]), I_qubit)

θ = π/2.5
RZ = kron(Rz(N, Array[[1,3]], θ), I_qubit)
RX = kron(Rx(N, Array[[3,4]], θ/4), I_qubit)
Udue = kron(U2(N, Array[[3,4]], [θ/2 θ/3 θ/4] ), I_qubit)

sx = kron(I_cavity, [0 1; 1 0])
sy = kron(I_cavity, [0 -1im; 1im 0])
# generating matrices
a,adag,sp,sm,sz = generate_matrices(N)

ψ_initial = zeros(N)
ψ_initial[1] = 1
ψ_initial = kron(ψ_initial/norm(ψ_initial), [0,1])

ψ_initial_rand = rand(N)
ψ_initial_rand = kron(ψ_initial_rand/norm(ψ_initial_rand), [0,1])
ψ_final_u = Udue * ψ_initial_rand
ψ_final_x = RX * ψ_initial_rand
ψ_final_x_gate = X_gate * ψ_initial_rand
ψ_final_superposition = kron(ones(N)/norm(ones(N)), [0,1])

ψ_final_k = zeros(N)
ψ_final_k[4] = 1
ψ_final_k = kron(ψ_final_k/norm(ψ_final_k), [0,1])


interaction_transformation(t) = cis(- (ω_c * a' * a + ω_q / 2 * sz) * t)
int_transformation = interaction_transformation(T)

8×8 Matrix{ComplexF64}:
 0.699251+0.714876im       0.0+0.0im       …       0.0+0.0im
      0.0+0.0im       0.699251-0.714876im          0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im       …       0.0+0.0im
      0.0+0.0im            0.0+0.0im               0.0+0.0im
      0.0+0.0im            0.0+0.0im          0.633413+0.773814im

In [43]:
H_drift = χ * adag * a * sz / 2

8×8 Matrix{Float64}:
 0.0  0.0  0.0    0.0   0.0   0.0  0.0    0.0
 0.0  0.0  0.0    0.0   0.0   0.0  0.0    0.0
 0.0  0.0  0.15   0.0   0.0   0.0  0.0    0.0
 0.0  0.0  0.0   -0.15  0.0   0.0  0.0    0.0
 0.0  0.0  0.0    0.0   0.3   0.0  0.0    0.0
 0.0  0.0  0.0    0.0   0.0  -0.3  0.0    0.0
 0.0  0.0  0.0    0.0   0.0   0.0  0.45   0.0
 0.0  0.0  0.0    0.0   0.0   0.0  0.0   -0.45

In [44]:
function cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, δt, coefficients, unitary, ω_c, ω_q, initial_state, final_state, amp_q, amp_c, int_transformation)

    # initialising the propagator
    dim = size(H_drift,1)
    propagator = Matrix{ComplexF64}(I,dim,dim)
    amplitude_c(t) = chebyshev_amplitude(coefficients[1:Int(length(coefficients)/2)], T, t)
    amplitude_q(t) = chebyshev_amplitude(coefficients[Int(length(coefficients)/2) + 1:end], T, t)

    # time ordered product of the single exponential matrices
    # is this true at all? or we need more time steps aniway?
    for l in 0:δt:T
        H = H_drift + amp_q * (amplitude_q(l) * sp + amplitude_q(l)' * sm) + amp_c * (amplitude_c(l) * a +  amplitude_c(l)' * adag)
        infinitesimal_propagator  = cis(- H * δt)
        propagator = infinitesimal_propagator * propagator
    end

    c = final_state' * int_transformation' * propagator * int_transformation * initial_state
   
    return 1 - norm(c)^2
end

function cost_from_0_dispersive_gate(H_drift, sp, sm, a, adag, T, δt, coefficients, unitary, ω_c, ω_q, amp_q, amp_c)

    # initialising the propagator
    dim = size(H_drift,1)
    propagator = Matrix{ComplexF64}(I,dim,dim)
    amplitude_c(t) = chebyshev_amplitude(coefficients[1:Int(length(coefficients)/2)], T, t)
    amplitude_q(t) = chebyshev_amplitude(coefficients[Int(length(coefficients)/2) + 1:end], T, t)

    # time ordered product of the single exponential matrices
    # is this true at all? or we need more time steps aniway?
    for l in 0:δt:T
        H = H_drift + amp_q * (amplitude_q(l) * sp + amplitude_q(l)' * sm) + amp_c * (amplitude_c(l) * a +  amplitude_c(l)' * adag)
        infinitesimal_propagator  = cis(- H * δt)
        propagator = infinitesimal_propagator * propagator
    end

    c = tr(unitary' * int_transformation * propagator)/dim
   
    return 1 - norm(c)^2
end

function cost_from_0_dispersive_gate_non_interaction(H_drift, sp, sm, a, adag, T, δt, coefficients, unitary, ω_c, ω_q, amp_q, amp_c)

    # initialising the propagator
    dim = size(H_drift,1)
    propagator = Matrix{ComplexF64}(I,dim,dim)
    amplitude_c(t) = chebyshev_amplitude(coefficients[1:Int(length(coefficients)/2)], T, t)
    amplitude_q(t) = chebyshev_amplitude(coefficients[Int(length(coefficients)/2) + 1:end], T, t)

    # time ordered product of the single exponential matrices
    # is this true at all? or we need more time steps aniway?
    for l in 0:δt:T
        H = H_drift + amp_q * (amplitude_q(l) * sp + amplitude_q(l)' * sm) + amp_c * (amplitude_c(l) * a +  amplitude_c(l)' * adag)
        infinitesimal_propagator  = cis(- H * δt)
        propagator = infinitesimal_propagator * propagator
    end

    c = tr(unitary' * propagator)/dim
   
    return 1 - norm(c)^2
end

function propagator(H_drift, sp, sm, a, adag, T, δt, coefficients)

    # initialising the propagator
    dim = size(H_drift,1)
    propagator = Matrix{ComplexF64}(I,dim,dim)

    amplitude_c = [ChebyshevT(coefficients[1:Int(length(coefficients)/2)])((2t - T)/T) for t in 0:δt:T]
    amplitude_q = [ChebyshevT(coefficients[Int(length(coefficients)/2) + 1:end])((2t - T)/T) for t in 0:δt:T]


    # time ordered product of the single exponential matrices
    # is this true at all? or we need more time steps aniway?
    for n in 1:1:Int(T/δt)+1
        H = H_drift + (amplitude_q(n) * sp + amplitude_q(n)' * sm) + (amplitude_c(n) * a + amplitude_c(n)' * adag)
        infinitesimal_propagator  = cis(- H * δt)
        propagator = infinitesimal_propagator * propagator
    end

    return propagator * exp(-1im*angle(propagator[8, 8]))
    
end

function gradient(coefficients, unitary, N, h, cost_before_increment,cost)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost(coeffs_copy) - cost_before_increment) / h
    end

    return gradient

end

gradient (generic function with 2 methods)

In [45]:
snap(x) = kron(Diagonal(exp.(-1im.*x)),I_qubit)
snap_params = rand(N) * 2 * pi .- pi
SNAP_GATE = snap(snap_params)

8×8 Matrix{ComplexF64}:
 -0.955251-0.295796im       -0.0-0.0im       …      0.0+0.0im
      -0.0-0.0im       -0.955251-0.295796im         0.0+0.0im
       0.0+0.0im             0.0+0.0im              0.0+0.0im
       0.0+0.0im             0.0+0.0im              0.0+0.0im
       0.0+0.0im             0.0+0.0im              0.0+0.0im
       0.0+0.0im             0.0+0.0im       …      0.0+0.0im
       0.0+0.0im             0.0+0.0im              0.0+0.0im
       0.0+0.0im             0.0+0.0im          0.94517+0.326578im

In [46]:
x(x) = cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, T / (50) , x, X_gate, ω_c, ω_q, ψ_initial_rand, ψ_final_x_gate, 5, 5, int_transformation)
rx(x) = cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, T / (50) , x, X_gate, ω_c, ω_q, ψ_initial_rand, ψ_final_x, 5, 5, int_transformation)
u(x) = cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, T / (50) , x, X_gate, ω_c, ω_q, ψ_initial_rand, ψ_final_u, 5, 5, int_transformation)
k(x) = cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, T / (50) , x, X_gate, ω_c, ω_q, ψ_initial, ψ_final_k, 5, 5, int_transformation)
superposition(x) = cost_from_0_dispersive(H_drift, sp, sm, a, adag, T, T / (50) , x, X_gate, ω_c, ω_q, ψ_initial, ψ_final_superposition, 5, 5, int_transformation)

it_costs_x = []
coefficients_x = []
it_costs_rx = []
coefficients_rx = []
it_costs_u = []
coefficients_u = []
it_costs_superposition = []
coefficients_superposition = []
it_costs_k = []
coefficients_k = []
for iteration in 1:3
    println(" iterations :", iteration)
    coefficients = rand(16)*2 .-1 + (rand(16)*2im .-1im)
    res_x = Optim.optimize(x, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=10))
    res_rx = Optim.optimize(rx, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=10))
    res_u = Optim.optimize(u, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=10))
    res_superposition = Optim.optimize(superposition, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=10))
    res_k = Optim.optimize(k, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=10))
    push!(it_costs_x,Optim.minimum(res_x))
    push!(it_costs_rx,Optim.minimum(res_rx))
    push!(it_costs_u,Optim.minimum(res_u))
    push!(it_costs_k,Optim.minimum(res_k))
    push!(it_costs_superposition,Optim.minimum(res_superposition))
    push!(coefficients_x,Optim.minimizer(res_x))
    push!(coefficients_rx,Optim.minimizer(res_rx))
    push!(coefficients_u,Optim.minimizer(res_u))
    push!(coefficients_k,Optim.minimizer(res_k))
    push!(coefficients_superposition,Optim.minimizer(res_superposition))
    println(Optim.minimum(res_k), Optim.minimum(res_x), Optim.minimum(res_rx), Optim.minimum(res_u), Optim.minimum(res_superposition))
end

 iterations :1
Iter     Function value   Gradient norm 


     0     3.908466e-01     4.210550e-01
 * time: 0.0002338886260986328


    10     1.086618e-02     2.076775e-04
 * time: 0.7955880165100098


    20     1.029453e-02     7.106292e-03
 * time: 1.678067922592163


    30     6.115786e-03     5.581970e-03
 * time: 2.2525668144226074


...

   350     8.591519e-10     1.772020e-06
 * time: 29.86220097541809


   360     4.163514e-11     8.622019e-08
 * time: 30.676843881607056


   370     3.717737e-11     1.244009e-07
 * time: 31.467566967010498


   380     1.629274e-11     7.608868e-08
 * time: 32.36912703514099


1.6211920694786386e-114.098765771232138e-113.3763556395172145e-91.6744361452936118e-100.005181191072029301


In [47]:
df_x = DataFrame(fidelity=it_costs_x, coefficients=coefficients_x)
df_rx = DataFrame(fidelity=it_costs_rx, coefficients=coefficients_rx)
df_u = DataFrame(fidelity=it_costs_u, coefficients=coefficients_u)
df_k = DataFrame(fidelity=it_costs_k, coefficients=coefficients_k)
df_superposition = DataFrame(fidelity=it_costs_superposition, coefficients=coefficients_superposition)
CSV.write("../../data/state_prep/3_photons/PO/x_gate_50steps_100ns.txt", df_x)
CSV.write("../../data/state_prep/3_photons/PO/rx_gate_50steps_100ns.txt", df_rx)
CSV.write("../../data/state_prep/3_photons/PO/u_gate_50steps_100ns.txt", df_u)
CSV.write("../../data/state_prep/3_photons/PO/k_gate_50steps_100ns.txt", df_k)
CSV.write("../../data/state_prep/3_photons/PO/superposition_gate_50steps_100ns.txt", df_superposition)

"../../data/state_prep/3_photons/PO/superposition_gate_50steps_100ns.txt"